In [185]:
pip install duckdb


In [187]:
import duckdb
con = duckdb.connect("carbon_offsets.duckdb")
con.execute("""
DROP TABLE IF EXISTS offsets_stage_v2;
DROP TABLE IF EXISTS offsets_clean_v2;
""")

In [189]:
con.execute("""
CREATE TEMP TABLE raw_offsets_v2 AS
SELECT *
FROM read_csv_auto(
  'combined_data.csv',
  HEADER = TRUE,
  SAMPLE_SIZE = 100000
);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [191]:
con.execute("""DESCRIBE raw_offsets_v2;
""").df()



,column_name,column_type,null,key,default,extra
0,Registry,VARCHAR,YES,None,None,None
1,Project_Identifier,VARCHAR,YES,None,None,None
2,Project_Title,VARCHAR,YES,None,None,None
3,Emission_Offset_Project_Developer,VARCHAR,YES,None,None,None
4,Quantification_Protocol,VARCHAR,YES,None,None,None
5,Vintage,BIGINT,YES,None,None,None
6,Serial_Start,VARCHAR,YES,None,None,None
7,Serial_End,VARCHAR,YES,None,None,None
8,Type,VARCHAR,YES,None,None,None
9,Quantity,BIGINT,YES,None,None,None


In [193]:
con.execute("""SELECT COUNT(*) AS raw_row_count FROM raw_offsets_v2;
""").df()


,raw_row_count
0,1048575


In [195]:
con.execute("""SELECT * FROM raw_offsets_v2 LIMIT 5;
""").df()

,Registry,Project_Identifier,Project_Title,Emission_Offset_Project_Developer,Quantification_Protocol,Vintage,Serial_Start,Serial_End,Type,Quantity,...,Facility,Emission_Offset_Status,Transaction_Id,Activity_Start,Offset_Start_Date,Offset_End_Date,Expiry_Date,Verifier_Company,Compliance_Year,Effective_Date
0,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,2007,8626-3432-000-000-070-058,8626-3432-000-000-107-037,None,36980,...,None,Revoked,NaN,NaT,2002-01-01,2007-12-31,2021-07-05,None,<NA>,2024-01-01
1,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,2006,8626-3432-000-000-055-706,8626-3432-000-000-070-057,None,14352,...,None,Revoked,NaN,NaT,2002-01-01,2007-12-31,2021-07-05,None,<NA>,2024-01-01
2,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,2005,8626-3432-000-000-041-500,8626-3432-000-000-055-705,None,14206,...,None,Revoked,NaN,NaT,2002-01-01,2007-12-31,2021-07-05,None,<NA>,2024-01-01
3,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,2004,8626-3432-000-000-013-653,8626-3432-000-000-027-541,None,13889,...,None,Revoked,NaN,NaT,2002-01-01,2007-12-31,2021-07-05,None,<NA>,2024-01-01
4,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,2003,8626-3432-000-000-027-542,8626-3432-000-000-041-499,None,13958,...,None,Revoked,NaN,NaT,2002-01-01,2007-12-31,2021-07-05,None,<NA>,2024-01-01


In [197]:
con.execute("""SELECT Registry, COUNT(*) AS n
FROM raw_offsets_v2
GROUP BY 1
ORDER BY n DESC
LIMIT 10;
""").df()

,Registry,n
0,AEOR,1048575


In [199]:
con.execute("""CREATE TABLE offsets_stage_v2 AS
SELECT
  UPPER(TRIM(CAST(Registry AS VARCHAR))) AS registry,
  TRIM(CAST(Project_Identifier AS VARCHAR)) AS project_id,
  TRIM(CAST(Project_Title AS VARCHAR)) AS project_title,
  TRIM(CAST(Emission_Offset_Project_Developer AS VARCHAR)) AS developer,

  REPLACE(REPLACE(TRIM(CAST(Quantification_Protocol AS VARCHAR)), 'Â', ''), '�', '') AS quantification_protocol,

  NULLIF(TRIM(CAST(Type AS VARCHAR)), '') AS project_type,
  TRY_CAST(NULLIF(TRIM(CAST(Vintage AS VARCHAR)), '') AS INTEGER) AS vintage,
  TRY_CAST(NULLIF(TRIM(CAST(Quantity AS VARCHAR)), '') AS BIGINT) AS quantity,

  NULLIF(TRIM(CAST(Current_Owner AS VARCHAR)), '') AS current_owner,
  NULLIF(TRIM(CAST(Facility AS VARCHAR)), '') AS facility,

  NULLIF(TRIM(CAST(Emission_Offset_Status AS VARCHAR)), '') AS offset_status,
  NULLIF(TRIM(CAST(Transaction_Id AS VARCHAR)), '') AS transaction_id,

  -- Keep serials as strings
  NULLIF(TRIM(CAST(Serial_Start AS VARCHAR)), '') AS serial_start,
  NULLIF(TRIM(CAST(Serial_End AS VARCHAR)), '') AS serial_end,

  NULLIF(TRIM(CAST(Verifier_Company AS VARCHAR)), '') AS verifier_company,
  TRY_CAST(NULLIF(TRIM(CAST(Compliance_Year AS VARCHAR)), '') AS INTEGER) AS compliance_year,

  -- Raw date strings (parse next step)
  NULLIF(TRIM(CAST(Activity_Start AS VARCHAR)), '') AS activity_start_raw,
  NULLIF(TRIM(CAST(Offset_Start_Date AS VARCHAR)), '') AS offset_start_raw,
  NULLIF(TRIM(CAST(Offset_End_Date AS VARCHAR)), '') AS offset_end_raw,
  NULLIF(TRIM(CAST(Expiry_Date AS VARCHAR)), '') AS expiry_raw,
  NULLIF(TRIM(CAST(Effective_Date AS VARCHAR)), '') AS effective_raw
FROM raw_offsets_v2;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [200]:
con.execute("""SELECT COUNT(*) AS stage_row_count FROM offsets_stage_v2;
""").df()

,stage_row_count
0,1048575


In [135]:
con.execute("""
SELECT
  SUM(CASE WHEN quantity IS NULL THEN 1 ELSE 0 END) AS missing_quantity,
  SUM(CASE WHEN quantity <= 0 THEN 1 ELSE 0 END) AS non_positive_quantity,
  MIN(quantity) AS min_quantity,
  MAX(quantity) AS max_quantity
FROM offsets_stage_v2;
""").df()

,missing_quantity,non_positive_quantity,min_quantity,max_quantity
0,0.0,20751.0,0,768138


In [137]:
con.execute("""
SELECT
  SUM(CASE WHEN project_type IS NULL THEN 1 ELSE 0 END) AS missing_type,
  SUM(CASE WHEN facility IS NULL THEN 1 ELSE 0 END) AS missing_facility,
  SUM(CASE WHEN transaction_id IS NULL THEN 1 ELSE 0 END) AS missing_transaction_id,
  SUM(CASE WHEN activity_start_raw IS NULL THEN 1 ELSE 0 END) AS missing_activity_start
FROM offsets_stage_v2;
""").df()

,missing_type,missing_facility,missing_transaction_id,missing_activity_start
0,1045359.0,1022894.0,739632.0,864695.0


In [141]:
con.execute("""
SELECT registry, COUNT(*) AS n
FROM offsets_stage_v2
GROUP BY 1
ORDER BY n DESC
LIMIT 10;
""").df()

,registry,n
0,AEOR,1048575


In [143]:
con.execute("""
CREATE TABLE offsets_clean_v2 AS
SELECT
  *,
  -- activity_start_dt: mostly missing, parse if available
  CASE
    WHEN activity_start_raw IS NULL THEN NULL
    WHEN REGEXP_MATCHES(activity_start_raw, '^\d{4}-\d{2}-\d{2}$') THEN TRY_CAST(activity_start_raw AS DATE)
    WHEN REGEXP_MATCHES(activity_start_raw, '^\d{1,2}/\d{1,2}/\d{4}$') THEN STRPTIME(activity_start_raw, '%m/%d/%Y')::DATE
    ELSE NULL
  END AS activity_start_dt,

  CASE
    WHEN offset_start_raw IS NULL THEN NULL
    WHEN REGEXP_MATCHES(offset_start_raw, '^\d{4}-\d{2}-\d{2}$') THEN TRY_CAST(offset_start_raw AS DATE)
    WHEN REGEXP_MATCHES(offset_start_raw, '^\d{1,2}/\d{1,2}/\d{4}$') THEN STRPTIME(offset_start_raw, '%m/%d/%Y')::DATE
    ELSE NULL
  END AS offset_start_dt,

  CASE
    WHEN offset_end_raw IS NULL THEN NULL
    WHEN REGEXP_MATCHES(offset_end_raw, '^\d{4}-\d{2}-\d{2}$') THEN TRY_CAST(offset_end_raw AS DATE)
    WHEN REGEXP_MATCHES(offset_end_raw, '^\d{1,2}/\d{1,2}/\d{4}$') THEN STRPTIME(offset_end_raw, '%m/%d/%Y')::DATE
    ELSE NULL
  END AS offset_end_dt,

  CASE
    WHEN expiry_raw IS NULL THEN NULL
    WHEN REGEXP_MATCHES(expiry_raw, '^\d{4}-\d{2}-\d{2}$') THEN TRY_CAST(expiry_raw AS DATE)
    WHEN REGEXP_MATCHES(expiry_raw, '^\d{1,2}/\d{1,2}/\d{4}$') THEN STRPTIME(expiry_raw, '%m/%d/%Y')::DATE
    ELSE NULL
  END AS expiry_dt,

  CASE
    WHEN effective_raw IS NULL THEN NULL
    WHEN REGEXP_MATCHES(effective_raw, '^\d{4}-\d{2}-\d{2}$') THEN TRY_CAST(effective_raw AS DATE)
    WHEN REGEXP_MATCHES(effective_raw, '^\d{1,2}/\d{1,2}/\d{4}$') THEN STRPTIME(effective_raw, '%m/%d/%Y')::DATE
    ELSE NULL
  END AS effective_dt
FROM offsets_stage_v2;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [147]:
con.execute("""
SELECT COUNT(*) AS clean_row_count FROM offsets_clean_v2;
""").df()

,clean_row_count
0,1048575


In [151]:
con.execute("""
SELECT * FROM offsets_clean_v2 LIMIT 5;
""").df()

,registry,project_id,project_title,developer,quantification_protocol,project_type,vintage,quantity,current_owner,facility,...,activity_start_raw,offset_start_raw,offset_end_raw,expiry_raw,effective_raw,activity_start_dt,offset_start_dt,offset_end_dt,expiry_dt,effective_dt
0,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,None,2007,36980,Heartland Generation Ltd.,None,...,None,2002-01-01,2007-12-31,2021-07-05,2024-01-01,NaT,2002-01-01,2007-12-31,2021-07-05,2024-01-01
1,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,None,2006,14352,Heartland Generation Ltd.,None,...,None,2002-01-01,2007-12-31,2021-07-05,2024-01-01,NaT,2002-01-01,2007-12-31,2021-07-05,2024-01-01
2,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,None,2005,14206,Heartland Generation Ltd.,None,...,None,2002-01-01,2007-12-31,2021-07-05,2024-01-01,NaT,2002-01-01,2007-12-31,2021-07-05,2024-01-01
3,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,None,2004,13889,Heartland Generation Ltd.,None,...,None,2002-01-01,2007-12-31,2021-07-05,2024-01-01,NaT,2002-01-01,2007-12-31,2021-07-05,2024-01-01
4,AEOR,8626-3432,2007 Tillage Auction Project,Emission Credits Corporation (ECC),Tillage System Management,None,2003,13958,Heartland Generation Ltd.,None,...,None,2002-01-01,2007-12-31,2021-07-05,2024-01-01,NaT,2002-01-01,2007-12-31,2021-07-05,2024-01-01


In [153]:
con.execute("""
SELECT
  MIN(offset_start_dt) AS min_offset_start,
  MAX(offset_start_dt) AS max_offset_start,
  MIN(offset_end_dt) AS min_offset_end,
  MAX(offset_end_dt) AS max_offset_end,
  MIN(expiry_dt) AS min_expiry,
  MAX(expiry_dt) AS max_expiry
FROM offsets_clean_v2;
""").df()

,min_offset_start,max_offset_start,min_offset_end,max_offset_end,min_expiry,max_expiry
0,2002-01-01,2024-05-28,2003-12-31,2039-02-25,2019-07-05,2033-07-05


In [157]:
con.execute("""
SELECT COUNT(*) AS invalid_offset_date_range
FROM offsets_clean_v2
WHERE offset_start_dt IS NOT NULL
  AND offset_end_dt IS NOT NULL
  AND offset_start_dt > offset_end_dt;
""").df()

,invalid_offset_date_range
0,0


In [159]:
con.execute("""
ALTER TABLE offsets_clean_v2 ADD COLUMN data_quality_flag VARCHAR;

-- Standardize status
UPDATE offsets_clean_v2
SET offset_status = UPPER(TRIM(offset_status))
WHERE offset_status IS NOT NULL;

UPDATE offsets_clean_v2
SET data_quality_flag =
  CASE
    WHEN quantity IS NULL OR quantity <= 0 THEN 'INVALID_QUANTITY'
    WHEN offset_start_dt IS NULL OR offset_end_dt IS NULL THEN 'MISSING_OFFSET_DATES'
    WHEN offset_start_dt > offset_end_dt THEN 'INVALID_DATE_RANGE'
    WHEN expiry_dt IS NOT NULL AND offset_end_dt IS NOT NULL AND expiry_dt < offset_end_dt THEN 'EXPIRY_BEFORE_END'
    WHEN vintage IS NOT NULL AND (vintage < 1990 OR vintage > 2050) THEN 'SUSPICIOUS_VINTAGE'
    ELSE 'VALID'
  END;
""")

In [171]:
con.execute("""
SELECT data_quality_flag, COUNT(*) AS n
FROM offsets_clean_v2
GROUP BY 1
ORDER BY n DESC;
""").df()

,data_quality_flag,n
0,VALID,854358
1,EXPIRY_BEFORE_END,173466
2,INVALID_QUANTITY,20751


In [175]:
con.execute("""
SELECT project_id, quantity, offset_start_dt, offset_end_dt, expiry_dt, data_quality_flag
FROM offsets_clean_v2
WHERE data_quality_flag <> 'VALID'
LIMIT 20;
""").df()

,project_id,quantity,offset_start_dt,offset_end_dt,expiry_dt,data_quality_flag
0,3434-3994,2582,2015-12-21,2023-12-20,2022-07-05,EXPIRY_BEFORE_END
1,3434-3994,3761,2015-12-21,2023-12-20,2022-07-05,EXPIRY_BEFORE_END
2,2691-7190,0,2016-01-01,2023-12-31,NaT,INVALID_QUANTITY
3,9273-8648,18159,2021-06-02,2031-06-01,2030-07-05,EXPIRY_BEFORE_END
4,9273-8648,946,2021-06-02,2031-06-01,2030-07-05,EXPIRY_BEFORE_END
5,9273-8648,7,2021-06-02,2031-06-01,2030-07-05,EXPIRY_BEFORE_END
6,9273-8648,9842,2021-06-02,2031-06-01,2030-07-05,EXPIRY_BEFORE_END
7,9273-8648,40,2021-06-02,2031-06-01,2030-07-05,EXPIRY_BEFORE_END
8,9623-5362,13473,2022-05-31,2032-05-30,2031-07-05,EXPIRY_BEFORE_END
9,7749-0748,0,2022-08-18,2032-08-17,NaT,INVALID_QUANTITY


In [181]:
df_clean = con.execute("SELECT * FROM offsets_clean_v2").df()
df_clean.to_csv("offsets_clean.csv", index=False)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [183]:
import os
print(os.getcwd())


C:\Users\shoaib
